In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from datetime import timedelta

# --- 1. Симуляция входных данных ---
# В реальности вы загрузите свои данные логов
# Предположим, у нас есть DataFrame `logs_df` с колонками:
# 'date' (тип datetime), 'category' (строка), 'count' (int)

print("1. Симуляция входных данных...")
dates = pd.date_range(
    start="2022-01-01", end="2024-12-29", freq="D"
)  # Почти 3 года данных
categories = ["cs", "math", "physics", "biology", "econ"]
data = []
for date in dates:
    for cat in categories:
        # Симулируем количество статей с некоторой сезонностью и шумом
        base_count = np.random.randint(5, 30)
        day_factor = max(
            0.1, np.sin(date.dayofweek * np.pi / 6) ** 2 + 0.5
        )  # Меньше на выходных
        year_factor = (
            1 + np.sin((date.dayofyear - 80) * 2 * np.pi / 365) * 0.2
        ) # Годовая сезонность
        trend_factor = 1 + (date - dates[0]).days / (3 * 365) * 0.1 # Небольшой тренд
        count = int(
            base_count * day_factor * year_factor * trend_factor + np.random.randn() * 3
        )
        data.append([date, cat, max(0, count)]) # Убедимся, что счетчик не отрицательный

logs_df = pd.DataFrame(data, columns=["date", "category", "count"])
print(f"Сгенерировано {len(logs_df)} записей логов.")
print(logs_df.head())

# --- 2. Агрегация данных по дням ---
# Суммируем статьи по всем категориям для каждого дня
print("\n2. Агрегация данных по дням...")
daily_totals_df = (
    logs_df.groupby("date")["count"].sum().reset_index()
)
daily_totals_df = daily_totals_df.rename(
    columns={"count": "total_daily_count"}
)
# Убедимся, что даты идут непрерывно (если есть пропуски, заполним нулями)
daily_totals_df = daily_totals_df.set_index("date").asfreq("D", fill_value=0).reset_index()
print(daily_totals_df.head())

# --- 3. Инжиниринг признаков (Feature Engineering) ---
print("\n3. Инжиниринг признаков...")

def create_features(df):
    """Создает признаки для модели на основе DataFrame с датой и счетчиком."""
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    df["dayofweek"] = df["date"].dt.dayofweek # 0 = Понедельник, 6 = Воскресенье
    df["dayofyear"] = df["date"].dt.dayofyear
    df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int) # Неделя года
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int) # Признак выходного дня

    # Лаговые признаки (значения из прошлого)
    # Используем значения за предыдущие дни и недели
    for lag in [1, 2, 3, 7, 14, 21, 28]:
        df[f"lag_{lag}"] = df["total_daily_count"].shift(lag)

    # Признаки скользящего окна (статистики за прошлые периоды)
    # Рассчитываем статистики по окну, *не включая текущий день* (используем shift(1))
    for window in [7, 14, 28]:
        rolling_window = df["total_daily_count"].shift(1).rolling(window=window)
        df[f"rolling_mean_{window}"] = rolling_window.mean()
        df[f"rolling_std_{window}"] = rolling_window.std()
        df[f"rolling_median_{window}"] = rolling_window.median()
        # Можно добавить min/max и другие статистики

    # Удаляем строки с NaN, появившиеся из-за лагов и скользящих окон
    df = df.dropna()
    return df

# Создаем признаки для исторических данных
features_df = create_features(daily_totals_df)
print(f"Созданы признаки. Размер данных: {features_df.shape}")
print(features_df.tail()) # Посмотрим на последние строки с признаками

# --- 4. Подготовка данных для модели ---
print("\n4. Подготовка данных для модели...")

# Целевая переменная - количество статей в день
TARGET = "total_daily_count"
# Признаки - все, кроме самой даты и целевой переменной
FEATURES = [
    col for col in features_df.columns if col not in ["date", TARGET]
]

print(f"Целевая переменная: {TARGET}")
print(f"Используемые признаки ({len(FEATURES)}): {FEATURES}")

# Разделение на обучающую и тестовую выборки делать не будем,
# т.к. будем использовать рекурсивный прогноз.
# Обучимся на всех доступных исторических данных.
X_train = features_df[FEATURES]
y_train = features_df[TARGET]

# --- 5. Обучение модели (LightGBM) ---
print("\n5. Обучение модели LightGBM...")

# Используем LightGBM Regressor
model = lgb.LGBMRegressor(
    objective="regression_l1", # MAE loss, можно 'regression' для MSE
    metric="mae",
    n_estimators=1000,       # Количество деревьев
    learning_rate=0.05,
    num_leaves=31,           # Стандартное значение
    max_depth=-1,            # Без ограничения глубины
    random_state=42,
    n_jobs=-1,               # Использовать все CPU
    reg_alpha=0.1,           # L1 регуляризация
    reg_lambda=0.1,          # L2 регуляризация
)

# Обучаем модель
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train)], # Можно добавить валидационный сет для ранней остановки
    eval_metric="mae",
    callbacks=[lgb.early_stopping(100, verbose=False)] # Ранняя остановка, если MAE не улучшается 100 раундов
)

print("Модель обучена.")

# --- 6. Рекурсивное прогнозирование на 8 недель (56 дней) ---
print("\n6. Рекурсивное прогнозирование на 8 недель...")

N_WEEKS_FORECAST = 8
FORECAST_HORIZON_DAYS = N_WEEKS_FORECAST * 7

# Последняя дата в наших данных с признаками
last_date = features_df["date"].max()
# Датафрейм для хранения истории (последние дни для расчета будущих признаков)
# Нам нужно достаточно данных для расчета максимального лага/окна
max_lag_or_window = 28 # Максимальное значение из лагов и окон
history_df = daily_totals_df[daily_totals_df['date'] <= last_date].tail(max_lag_or_window * 2) # Берем с запасом

future_predictions = []
current_date = last_date

for i in range(FORECAST_HORIZON_DAYS):
    current_date += timedelta(days=1)
    print(f"Прогнозируем на {current_date.strftime('%Y-%m-%d')} ({i+1}/{FORECAST_HORIZON_DAYS})")

    # Создаем строку с будущей датой для генерации признаков
    future_row_base = pd.DataFrame({'date': [current_date], 'total_daily_count': [np.nan]}) # Пока не знаем значение

    # Добавляем эту строку к истории, чтобы рассчитать признаки
    temp_history = pd.concat([history_df, future_row_base], ignore_index=True)

    # Генерируем признаки для *последней* строки (будущей даты)
    # Функция create_features удалит строки с NaN, но нам нужна только последняя
    # Поэтому сначала генерируем, потом берем последнюю строку
    all_features_temp = create_features(temp_history)
    future_features = all_features_temp.iloc[[-1]][FEATURES] # Берем последнюю строку и нужные колонки

    # Делаем предсказание на один шаг вперед
    prediction = model.predict(future_features)[0]
    # Округляем и делаем неотрицательным
    prediction = max(0, round(prediction))

    # Сохраняем предсказание
    future_predictions.append({'date': current_date, 'predicted_count': prediction})

    # Обновляем историю: добавляем предсказанное значение для использования на следующем шаге
    # Важно: используем предсказанное значение как 'total_daily_count' для будущих лагов/окон
    history_df = pd.concat([
        history_df,
        pd.DataFrame({'date': [current_date], 'total_daily_count': [prediction]})
    ], ignore_index=True)
    # Можно ограничить размер history_df, чтобы не рос бесконечно

# Преобразуем список предсказаний в DataFrame
future_preds_df = pd.DataFrame(future_predictions)
print("\nДневные прогнозы:")
print(future_preds_df)

# --- 7. Агрегация прогнозов по неделям ---
print("\n7. Агрегация прогнозов по неделям...")

# Устанавливаем дату как индекс для удобной ресемплинга
future_preds_df = future_preds_df.set_index('date')

# Ресемплим по неделям, заканчивающимся в воскресенье ('W-SUN') и суммируем
# Убедимся, что первая неделя начинается с первого дня прогноза
weekly_forecast = future_preds_df['predicted_count'].resample('W-SUN').sum()

# Оставляем только нужные 8 недель прогноза
weekly_forecast = weekly_forecast.head(N_WEEKS_FORECAST)

print(f"\nПрогноз суммарного количества статей на следующие {N_WEEKS_FORECAST} недель:")
print(weekly_forecast)

# --- 8. Визуализация (опционально) ---
print("\n8. Визуализация...")

plt.figure(figsize=(15, 7))

# Исторические данные (недельные суммы)
weekly_history = daily_totals_df.set_index('date')['total_daily_count'].resample('W-SUN').sum()
plt.plot(weekly_history.index, weekly_history.values, label='Исторические недельные суммы', color='blue')

# Предсказанные данные (недельные суммы)
plt.plot(weekly_forecast.index, weekly_forecast.values, label='Прогноз на 8 недель', color='red', marker='o')

plt.title('Прогноз еженедельного количества статей на arXiv')
plt.xlabel('Неделя')
plt.ylabel('Суммарное количество статей')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# ... (остальной код до функции create_features) ...

def create_features(df):
    """Создает признаки для модели на основе DataFrame с датой и счетчиком."""
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # --- Изменения для дня недели ---
    # Сначала получаем номер дня недели (0=Пн, 6=Вс)
    day_of_week_num = df["date"].dt.dayofweek
    # Применяем sin/cos преобразование
    df["dayofweek_sin"] = np.sin(2 * np.pi * day_of_week_num / 7)
    df["dayofweek_cos"] = np.cos(2 * np.pi * day_of_week_num / 7)
    # Оригинальный 'dayofweek' теперь можно не использовать как признак,
    # но он был нужен для расчета sin/cos.
    # ---------------------------------

    df["dayofyear"] = df["date"].dt.dayofyear
    df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
    df["month"] = df["date"].dt.month # Месяц тоже циклический! Можно применить sin/cos и к нему.
    # df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    # df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df["year"] = df["date"].dt.year
    # Признак выходного дня все еще может быть полезен сам по себе
    df["is_weekend"] = day_of_week_num.isin([5, 6]).astype(int)

    # Лаговые признаки (значения из прошлого)
    for lag in [1, 2, 3, 7, 14, 21, 28]:
        df[f"lag_{lag}"] = df["total_daily_count"].shift(lag)

    # Признаки скользящего окна (статистики за прошлые периоды)
    for window in [7, 14, 28]:
        rolling_window = df["total_daily_count"].shift(1).rolling(window=window)
        df[f"rolling_mean_{window}"] = rolling_window.mean()
        df[f"rolling_std_{window}"] = rolling_window.std()
        df[f"rolling_median_{window}"] = rolling_window.median()

    df = df.dropna()
    return df

# ... (остальной код) ...

# --- Обновление списка признаков ---
# Теперь нужно использовать новые признаки и убрать старый 'dayofweek'

# Создаем признаки для исторических данных
features_df = create_features(daily_totals_df)
print(f"Созданы признаки. Размер данных: {features_df.shape}")
print(features_df[['date', 'total_daily_count', 'dayofweek_sin', 'dayofweek_cos', 'is_weekend']].tail())

# Целевая переменная - количество статей в день
TARGET = "total_daily_count"
# Признаки - все, кроме самой даты и целевой переменной
# Убедимся, что 'dayofweek' (старый числовой) не попал в список, а новые sin/cos попали
FEATURES = [
    col for col in features_df.columns if col not in ["date", TARGET]
]
# Если вы хотите явно убедиться, что старый 'dayofweek' не используется:
# FEATURES = [
#     'dayofweek_sin', 'dayofweek_cos', 'dayofyear', 'weekofyear', 'month', 'year', 'is_weekend',
#     'lag_1', 'lag_2', 'lag_3', 'lag_7', 'lag_14', 'lag_21', 'lag_28',
#     'rolling_mean_7', 'rolling_std_7', 'rolling_median_7',
#     'rolling_mean_14', 'rolling_std_14', 'rolling_median_14',
#     'rolling_mean_28', 'rolling_std_28', 'rolling_median_28'
#     # Добавьте сюда 'month_sin', 'month_cos', если решите их использовать
# ]


print(f"Целевая переменная: {TARGET}")
print(f"Используемые признаки ({len(FEATURES)}): {FEATURES}")

# --- Далее обучение модели и прогнозирование как раньше ---
# Модель будет использовать 'dayofweek_sin' и 'dayofweek_cos' вместо 'dayofweek'
# ... (код обучения и прогнозирования) ...

Агрегация данных:


Поскольку цель - предсказать суммарное количество статей за неделю по всем категориям, первым делом нужно агрегировать ваши ежедневные данные по категориям.
Для каждой даты сложите количество статей \(Z\) по всем категориям \(Y\). У вас получится новый временной ряд: (Дата, Общее_Число_Статей_за_День).
Далее, поскольку прогноз нужен на неделю, можно либо работать с дневными данными и потом суммировать предсказания за 7 дней, либо сразу агрегировать данные до недельного уровня: (Номер_Недели, Суммарное_Число_Статей_за_Неделю). Я бы, вероятно, начал с дневных данных, так как они содержат больше информации о внутринедельной динамике, которая может быть полезна.



Инжиниринг признаков (Feature Engineering): Это ключевой этап для не-рекуррентных моделей (как деревья решений или линейные модели). На основе временного ряда (Дата, Общее_Число_Статей_за_День) создаем признаки для каждой даты \(t\), которые помогут предсказать будущее:


Календарные признаки:

День недели (1-7): Очень важно, так как активность публикаций сильно зависит от дня (например, меньше в выходные).
День месяца, день года.
Неделя года.
Месяц, год: Для улавливания годовой сезонности и долгосрочных трендов.
Признак выходного/праздничного дня: Может сильно влиять на количество публикаций. Потребуется внешний календарь праздников.
Возможно, признаки, связанные с крупными конференциями (например, "неделя перед дедлайном конференции X"): Это сложнее, требует внешних данных, но может дать сильный сигнал.


Лаговые признаки (Lag Features): Значения ряда из прошлого.

Количество статей вчера (\(\text{count}_{t-1}\)).
Количество статей 7 дней назад (\(\text{count}_{t-7}\)): Для улавливания недельной сезонности.
Количество статей 14, 21, 28 дней назад (\(\text{count}_{t-14}, \text{count}_{t-21}, \text{count}_{t-28}\)).
Можно добавить и другие лаги (2, 3, ... дней назад).


Признаки скользящего окна (Rolling Window Features): Статистики по предыдущим периодам.

Среднее количество статей за последние 7 дней.
Среднее количество статей за последние 14, 28 дней.
Медиана, стандартное отклонение, min/max за последние N дней (например, N=7, 14, 28). Это помогает уловить недавний тренд и волатильность.


(Опционально) Можно попробовать добавить признаки, основанные на категориях, даже если цель - общий итог. Например, доля статей по CS за вчерашний день, или скользящее среднее по ключевым категориям.



Выбор стратегии моделирования (для прогноза на 8 недель):



Рекурсивная стратегия:


Обучаем модель предсказывать один шаг вперед (например, количество статей на следующий день).
Чтобы предсказать на 8 недель (56 дней), делаем 56 предсказаний: предсказываем день 1, используем это предсказание для генерации признаков для дня 2, предсказываем день 2, и так далее.
Затем суммируем предсказанные дневные значения по каждой из 8 будущих недель.
Плюс: Используется одна модель.
Минус: Ошибки могут накапливаться с каждым шагом прогноза.



Прямая (Direct) стратегия:


Обучаем 8 разных моделей. Модель 1 предсказывает суммарное количество за 1-ю неделю, Модель 2 - за 2-ю неделю, ..., Модель 8 - за 8-ю неделю.
Каждая модель использует признаки, доступные на момент прогнозирования (т.е. исторические данные до начала 1-й недели). Целевой переменной для модели \(k\) будет суммарное число статей за \(k\)-ю неделю в будущем.
Плюс: Нет проблемы накопления ошибок.
Минус: Нужно обучать и поддерживать 8 моделей. Может не улавливать зависимость между прогнозами на разные недели.



Seq2Seq модели (например, LSTM, GRU, Transformer):


Подаем на вход последовательность прошлых данных (например, дневные или недельные количества статей и признаки за последние N недель/дней).
Модель обучается напрямую выдавать последовательность из 8 значений - прогнозы на следующие 8 недель.
Плюс: Элегантно решает задачу многошагового прогнозирования, может улавливать сложные временные зависимости.
Минус: Более сложны в реализации и настройке, требуют больше данных и вычислительных ресурсов.



Рекомендация: Я бы начал с рекурсивной стратегии, используя дневные данные и суммируя их потом по неделям. Она часто дает хороший результат и проще в реализации, чем Seq2Seq. Если накопление ошибки станет проблемой, можно попробовать прямую стратегию или Seq2Seq.





Выбор модели (для рекурсивной или прямой стратегии):


Градиентный бустинг (LightGBM, XGBoost, CatBoost): Обычно показывают отличные результаты на табличных данных с хорошо подготовленными признаками (как в нашем случае с лагами и скользящими средними). Часто лучший выбор для таких задач.
Random Forest: Тоже хороший вариант, менее чувствителен к настройке гиперпараметров, чем бустинг.
Линейные модели (Linear Regression, Ridge, Lasso): Хороший бейзлайн, быстры в обучении, интерпретируемы. Могут хорошо работать, если зависимости близки к линейным и признаки подобраны удачно.
Классические модели временных рядов (SARIMA, Prophet): Можно использовать их на агрегированном недельном ряде как бейзлайн или как часть ансамбля. Prophet хорошо работает с человеческой активностью и сезонностью.



Обучение и Валидация:


Важно: Нельзя использовать стандартную кросс-валидацию со случайным перемешиванием! Данные имеют временную структуру.
Используйте временную валидацию (Time Series Split / Walk-Forward Validation): Обучаетесь на данных до точки \(T\), валидируетесь на данных с \(T\) до \(T+k\). Затем сдвигаете окно: обучаетесь до \(T+k\), валидируетесь с \(T+k\) до \(T+2k\), и так далее.
Метрики: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error), MAPE/sMAPE (Mean Absolute Percentage Error / symmetric MAPE) - выбирайте в зависимости от того, что важнее (абсолютная ошибка, относительная, штраф за большие ошибки).



Прогнозирование:


Обучите финальную модель на всех доступных исторических данных.
Сгенерируйте прогнозы на следующие 8 недель, используя выбранную стратегию (рекурсивную, прямую или Seq2Seq).